In [22]:
from game_logic.ghostAgents import RandomGhost
from entregables.maxNAgent import MaxNAgent
from game_logic.randomPacman import RandomPacman
from game_logic.PacmanEnvAbs import PacmanEnvAbs
import random
import math
import numpy as np
from game_logic.game_util import process_state
from main import get_default_agents
import timeit 
import pandas as pd

all_layouts = [
        "custom1",
        "custom2",
        "capsuleClassic",
        "contestClassic",
        "mediumClassic",
        "minimaxClassic",
        "openClassic",
        "originalClassic",
        "smallClassic",
        "testClassic",
        "trappedClassic",
        "trickyClassic",
        "mediumGrid",
        "smallGrid"
    ]

# Parametros para un test
class TestParams():
    def __init__(self, test_name, layout, pacman_agent, ghost_agent_0, ghost_agent_1):        
        self.test_name = test_name
        self.layout = layout
        self.pacman_agent = pacman_agent
        self.ghost_agent_0 = ghost_agent_0
        self.ghost_agent_1 = ghost_agent_1


In [23]:
def run_test(test_params):  
    t0 = timeit.default_timer()   
    pacman_agent = test_params.pacman_agent
    ghost_agent_0 = test_params.ghost_agent_0
    ghost_agent_1 = test_params.ghost_agent_1
    agents = [pacman_agent, ghost_agent_0, ghost_agent_1]
    agents.extend(get_default_agents(3, 10))    
    done = False
    env = PacmanEnvAbs(agents = agents, view_distance = (2, 2))      
    game_state = env.reset(enable_render= False, layout_name= test_params.layout)
    turn_index = 0
    while (not(done)): # jugar   
        action = agents[turn_index].getAction(game_state)
        game_state, rewards, done, info = env.step(action, turn_index)        
        turn_index = (turn_index + 1) % env._get_num_agents()  
    t1 = timeit.default_timer()    
    time = t1-t0  
    assert(game_state.getNumAgents()>=2) # que el juego tenga mas de 2 agentes
    if game_state.getNumAgents()==2: # vector de rewards con los primeros 2 rewards y nan
        ret = game_state.get_rewards()
        ret.append(np.nan)
    else: # vector de rewards con los primeros 3 rewards
        ret = game_state.get_rewards()[0:3]
    return ret, time

In [24]:
pacman_agent = RandomPacman(index = 0)
ghost_agent_0 = MaxNAgent(index = 1, unroll_type="MCTS", max_unroll_depth=12, number_of_unrolls=6)
ghost_agent_1 = RandomGhost(index = 2)
sample_test = TestParams("PrimerTest", "mediumGrid", pacman_agent, ghost_agent_0, ghost_agent_1)
print(run_test(sample_test))


([-1523, -5646.0, nan], 1.4935193139999683)


In [25]:
selected_layouts = [  
        "smallGrid"
    ]
# 1 pacman:
    # MCTS --> Eval 1 (get_rewards)
pacmans = [
    ("MAXN_pcmn",MaxNAgent(index = 0, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10))
]

# 6 ghosts:
    # MC ---> Eval 1 (get_rewards)
    # MC ---> Eval 2 (get_rewards)
    # MC ---> Eval 3 (get_rewards)
    # MC ---> Eval 1 (use_eval_in_end)
    # MC ---> Eval 2 (use_eval_in_end)
    # MC ---> Eval 3 (use_eval_in_end)    
ghosts0s = [
    ("Eval1_MC_gh0", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=1, use_eval_in_end=False)),
    ("Eval2_MC_gh0", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=2, use_eval_in_end=False)),
    ("Eval3_MC_gh0", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=3, use_eval_in_end=False)),
    ("Eval1_MC_gh0_isEval", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=1, use_eval_in_end=True)),
    ("Eval2_MC_gh0_isEval", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=2, use_eval_in_end=True)),
    ("Eval3_MC_gh0_isEval", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=3, use_eval_in_end=True))    
]

# 6 ghosts:
    # MCTS ---> Eval 1 (get_rewards)
    # MCTS ---> Eval 2 (get_rewards)
    # MCTS ---> Eval 3 (get_rewards)
    # MCTS ---> Eval 1 (use_eval_in_end)
    # MCTS ---> Eval 2 (use_eval_in_end)
    # MCTS ---> Eval 3 (use_eval_in_end) 
ghosts1s = [
    ("Eval1_MCTS_gh1", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=1, use_eval_in_end=False)),
    ("Eval2_MCTS_gh1", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=2, use_eval_in_end=False)),
    ("Eval3_MCTS_gh1", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=3, use_eval_in_end=False)),
    ("Eval1_MCTS_gh1_isEval", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=1, use_eval_in_end=True)),
    ("Eval2_MCTS_gh1_isEval", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=2, use_eval_in_end=True)),
    ("Eval3_MCTS_gh1_isEval", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=3, use_eval_in_end=True))    
]

In [26]:
all_tests = []
for layout in selected_layouts:
    for pacman_name, pacman_agent in pacmans:
        for ghost0_name, ghost_0 in ghosts0s:
            for ghost1_name, ghost_1 in ghosts1s:
                name = layout+"_"+pacman_name+"_"+ghost0_name+"_"+ghost1_name
                test = TestParams(name, layout, pacman_agent, ghost_agent_0, ghost_agent_1)
                all_tests.append(test)
number_of_runs = 10

In [28]:
results = []
t0 = timeit.default_timer()   
for i in range(len(all_tests)):   
        values = []
        times = []    
        for _ in range(number_of_runs):
            value, time = run_test(all_tests[i])            
            values.append(value)
            times.append(time)
        #compute avg and std and and add to results
        value_avg = np.round(np.mean(np.array(values), axis=0),2)
        time_avg = np.round(np.mean(np.array(times)),2)
        value_std = np.round(np.std(np.array(values), axis=0),2)
        time_std = np.round(np.std(np.array(times)),2)
        result = {'TestName':all_tests[i].test_name,'Time_Avg':time_avg,'Time_Std':time_std, 
                  'P_Value_Avg':value_avg[0], 
                  'P_Value_Std':value_std[0],
                  'GH0_Value_Avg':value_avg[1], 
                  'GH0_Value_Std':value_std[1],
                  'GH1_Value_Avg':value_avg[2], 
                  'GH1_Value_Std':value_std[2]
                  }
        results.append(result)
results = pd.DataFrame(results)
t1 = timeit.default_timer()    
time = t1-t0  
print("All tests time: ", time, "seconds.")  
pd.set_option('display.max_rows', 100)  
display(results)

All tests time:  524.800428085 seconds.


,TestName,Time_Avg,Time_Std,P_Value_Avg,P_Value_Std,GH0_Value_Avg,GH0_Value_Std,GH1_Value_Avg,GH1_Value_Std
0,smallGrid_MAXN_pcmn_Eval1_MC_gh0_Eval1_MCTS_gh1,1.75,1.51,-1324.7,609.46,-5597.6,5744.76,NaN,NaN
1,smallGrid_MAXN_pcmn_Eval1_MC_gh0_Eval2_MCTS_gh1,1.03,0.84,-1517.0,13.74,-2875.9,3140.06,NaN,NaN
2,smallGrid_MAXN_pcmn_Eval1_MC_gh0_Eval3_MCTS_gh1,1.81,1.32,-720.8,988.45,-6023.6,4912.02,NaN,NaN
3,smallGrid_MAXN_pcmn_Eval1_MC_gh0_Eval1_MCTS_gh...,1.73,2.97,-1326.7,611.62,-5346.7,10493.81,NaN,NaN
4,smallGrid_MAXN_pcmn_Eval1_MC_gh0_Eval2_MCTS_gh...,1.43,1.06,-1118.2,800.75,-4505.9,4113.72,NaN,NaN
5,smallGrid_MAXN_pcmn_Eval1_MC_gh0_Eval3_MCTS_gh...,1.81,1.19,-1126.5,814.70,-5924.6,4302.66,NaN,NaN
6,smallGrid_MAXN_pcmn_Eval2_MC_gh0_Eval1_MCTS_gh1,1.21,0.69,-1116.7,806.94,-3766.8,2733.91,NaN,NaN
7,smallGrid_MAXN_pcmn_Eval2_MC_gh0_Eval2_MCTS_gh1,0.65,0.41,-1108.2,801.85,-1679.0,1947.87,NaN,NaN
8,smallGrid_MAXN_pcmn_Eval2_MC_gh0_Eval3_MCTS_gh1,2.62,3.57,-1138.6,773.79,-9125.3,13722.11,NaN,NaN
9,smallGrid_MAXN_pcmn_Eval2_MC_gh0_Eval1_MCTS_gh...,2.01,1.87,-926.0,926.46,-6875.7,7040.75,NaN,NaN
